**This is the codebook about 1 VS N labelled classification.   
The methods we used: XGBoost, new feature(word count) and oversample.  
During the modeling, we involved random search to tuning the hyper parameters.  
Finally, the XGBoost with word count perform best, which obtain acc 0.84 and F1 score (0.9 and 0.64).**

In [1]:
import numpy as np
import pandas as pd 
from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import GridSearchCV 

In [2]:
df = pd.read_csv('df_prep2.csv')

In [3]:
df1 = df.copy()
l1 = []
for i in range(df1.shape[0]):
    if df1['risk_cat'][i]=='Not Risk':
        l1.append(1)
    else:
        l1.append(0)
df1['Not Risk'] = l1

x_train, x_test, y_train, y_test = model_selection.train_test_split(df1.contents,df1['Not Risk'],
                                                                    test_size = 0.2, random_state = 42) 
tfidf1 = TfidfVectorizer() 
tfidf1.fit(x_train)  
x_train= tfidf1.transform(x_train) 
x_test = tfidf1.transform(x_test) 
df1['word_count'] = [len(item.split(" ")) for item in df1['contents']]
x_train_wc =  hstack([x_train,np.array([df1.iloc[y_train.index.values]['word_count']]).T])
x_test_wc =  hstack([x_test,np.array([df1.iloc[y_test.index.values]['word_count']]).T])

XGBoost + Wordcount

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
clf0 = XGBClassifier()

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

random_cv=RandomizedSearchCV(estimator=clf0,param_distributions=params,
                             cv=5,n_iter=5,scoring='roc_auc_ovo_weighted',n_jobs=1,verbose=3,return_train_score=True,random_state=121)
random_cv.fit(x_train_wc,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:33:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.980, test=0.878), total=   4.6s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s remaining:    0.0s
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:33:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.983, test=0.884), total=   4.5s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


...

C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.8, min_child_weight=1, max_depth=4, gamma=1, colsample_bytree=0.6, score=(train=0.997, test=0.902), total=   5.8s


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  2.5min finished
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:36:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [13]:
random_cv.best_params_

{'subsample': 0.8,
 'min_child_weight': 1,
 'max_depth': 4,
 'gamma': 1,
 'colsample_bytree': 0.6}

In [15]:
clf1 = XGBClassifier(
 subsample= 0.8,
 min_child_weight= 1,
 max_depth= 4,
 gamma= 1,
 colsample_bytree= 0.6)

clf1.fit(x_train_wc, y_train)

[10:43:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
print('train set mean accuracy %s' % clf1.score(x_train_wc,y_train))
pred0 = clf1.predict(x_test_wc)
print('accuracy %s' % accuracy_score(pred0, y_test))
print(classification_report(y_test, pred0))

train set mean accuracy 0.9749223787914975
accuracy 0.8433619866284623
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       803
           1       0.69      0.59      0.64       244

    accuracy                           0.84      1047
   macro avg       0.79      0.76      0.77      1047
weighted avg       0.84      0.84      0.84      1047



XGBoost without wordcount

In [21]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

random_cv=RandomizedSearchCV(estimator=clf0,param_distributions=params,
                             cv=5,n_iter=5,scoring='roc_auc_ovo_weighted',n_jobs=1,verbose=3,return_train_score=True,random_state=121)
random_cv.fit(x_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:44:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.981, test=0.878), total=   4.5s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.6s remaining:    0.0s
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:44:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.982, test=0.883), total=   4.5s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


...

C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:46:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.8, min_child_weight=1, max_depth=4, gamma=1, colsample_bytree=0.6, score=(train=0.997, test=0.898), total=   5.9s


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  2.4min finished
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:46:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [22]:
random_cv.best_params_

{'subsample': 0.8,
 'min_child_weight': 1,
 'max_depth': 4,
 'gamma': 1,
 'colsample_bytree': 0.6}

In [23]:
clf2 = XGBClassifier(
 subsample= 0.8,
 min_child_weight= 1,
 max_depth= 4,
 gamma= 1,
 colsample_bytree= 0.6)

clf2.fit(x_train, y_train)

[10:47:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
print('train set mean accuracy %s' % clf2.score(x_train,y_train))
pred1 = clf2.predict(x_test)
print('accuracy %s' % accuracy_score(pred1, y_test))
print(classification_report(y_test, pred1))

train set mean accuracy 0.9722951994267972
accuracy 0.8424068767908309
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       803
           1       0.69      0.58      0.63       244

    accuracy                           0.84      1047
   macro avg       0.79      0.75      0.77      1047
weighted avg       0.84      0.84      0.84      1047



XGboost with Wordcount and oversample

In [27]:
from imblearn.over_sampling import RandomOverSampler
os = RandomOverSampler(sampling_strategy='minority')
x_train_os, y_train_os = os.fit_sample(x_train_wc,y_train)
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

random_cv=RandomizedSearchCV(estimator=clf0,param_distributions=params,
                             cv=5,n_iter=5,scoring='roc_auc',n_jobs=1,verbose=3,return_train_score=True,random_state=121)
random_cv.fit(x_train_os,y_train_os)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:49:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.986, test=0.956), total=   6.2s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.4s remaining:    0.0s
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:49:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8, score=(train=0.987, test=0.946), total=   6.1s
[CV] subsample=0.6, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.8 


...

C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:52:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  subsample=0.8, min_child_weight=1, max_depth=4, gamma=1, colsample_bytree=0.6, score=(train=0.998, test=0.983), total=   7.7s


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  3.3min finished
C:\Users\E055761\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:52:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [28]:
random_cv.best_params_

{'subsample': 0.6,
 'min_child_weight': 1,
 'max_depth': 5,
 'gamma': 1.5,
 'colsample_bytree': 0.8}

In [29]:
clf3 = XGBClassifier(
 subsample= 0.6,
 min_child_weight= 1,
 max_depth= 5,
 gamma= 1.5,
 colsample_bytree= 0.8)

clf3.fit(x_train_os, y_train_os)

[10:54:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1.5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
print('train set mean accuracy %s' % clf3.score(x_train_os,y_train_os))
pred2 = clf3.predict(x_test_wc)
print('accuracy %s' % accuracy_score(pred2, y_test))
print(classification_report(y_test, pred2))

train set mean accuracy 0.9887041104486979
accuracy 0.8299904489016237
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       803
           1       0.62      0.69      0.65       244

    accuracy                           0.83      1047
   macro avg       0.76      0.78      0.77      1047
weighted avg       0.84      0.83      0.83      1047

